In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
names = ['id','time','extype','examount','mark']
bank_detail_train = pd.read_csv("../../pcredit/train/bank_detail_train.txt",header=None)
bank_detail_train.columns = names
bank_detail_train.head()

,id,time,extype,examount,mark
0,6965,5894316387,0,13.756664,0
1,6965,5894321388,1,13.756664,0
2,6965,5897553564,0,14.449810,0
3,6965,5897563463,1,10.527763,0
4,6965,5897564598,1,13.651303,0


In [9]:
df = pd.pivot_table(bank_detail_train,index=['id','extype'],values=['examount'],aggfunc=np.mean)
df.head()

examount
id extype           
1  0       13.734079
   1       13.255397
3  0       13.249264
   1        9.834236
4  0       12.128566

In [15]:
dfun = df.unstack()
dfun['sub'] = dfun['examount'][0]-dfun['examount'][1]
dfun['id'] = dfun.index
dfun.head()

examount                  sub  id
extype          0          1              
id                                        
1       13.734079  13.255397  0.478681   1
3       13.249264   9.834236  3.415028   3
4       12.128566  10.920132  1.208435   4
9       14.371789  12.867697  1.504092   9
10      12.720866  12.296680  0.424186  10

In [26]:
dd = dfun[['id','sub']]
dd['examount0'] = dfun['examount'][0]
dd['examount1'] = dfun['examount'][1]
print dd.index.names,dd.columns.names
print dd.head()

[u'id'] [None, u'extype']
        id       sub  examount0  examount1
extype                                    
id                                        
1        1  0.478681  13.734079  13.255397
3        3  3.415028  13.249264   9.834236
4        4  1.208435  12.128566  10.920132
9        9  1.504092  14.371789  12.867697
10      10  0.424186  12.720866  12.296680


/home/hadoop/.local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [18]:
dd.to_csv('../data/train/bank_train.csv',index=None)